# Warhammer calculations

In [1]:
from __future__ import absolute_import, division, print_function
from builtins import *
import scipy.stats as stats
import scipy.misc as scm
import matplotlib.pyplot as plt

## Function defines

In [2]:
def successes(tries, prob):
    return {x: round(stats.binom.pmf(x, tries, prob) * 1000) / 1000 for x in range(tries+1)}

In [3]:
def compound_successes(profile, prob):
    return {x: round(sum(stats.binom.pmf(x, y, prob) * profile[y] for y in profile if y >= x) * 1000) / 1000 for x in profile}

In [4]:
def get_damage(profile, damage_list):
    n = len(damage_list)
    if n == 1: return {x * damage_list[0]: profile[x] for x in profile if profile[x] > 0}
    output = {0: profile[0]} 
    for h in profile:
        if h == 0 or profile[h] < 0.005: continue
        for d in range(h * min(damage_list), max(damage_list) * h + 1):
            if d in output:
                output[d] += profile[h] * (1 / n ** h) * sum(((-1) ** i) * scm.comb(h, i) * scm.comb(d - n * i - 1, h - 1) 
                                                              for i in range(int((d - h) / n) + 1))
            else:
                output[d] = profile[h] * (1 / n ** h) * sum(((-1) ** i) * scm.comb(h, i) * scm.comb(d - n * i - 1, h - 1) 
                                                              for i in range(int((d - h) / n) + 1))
    return output    

In [5]:
damage_types = {'1': [1], '2': [2], '3': [3], 'd3': [1, 2, 3], 'd6': [1, 2, 3, 4, 5, 6]}

## Variable input

In [6]:
shots = 10
p_hit = 2 / 3
p_wound =  2 / 3
p_unsaved = 5 / 6
damage = 'd6'
target_wounds = 6

## Work

In [ ]:
damage_list = damage_types[damage]
hits = successes(shots, p_hit)
wounds = compound_successes(hits, p_wound)
unsaved = compound_successes(wounds, p_unsaved)
damage_dealt = get_damage(unsaved, damage_list)
for d in damage_dealt.copy():
    if d > target_wounds: 
        damage_dealt[target_wounds] += damage_dealt[d]
        del damage_dealt[d]

## Present

In [ ]:
fig = plt.figure()
plt.title('{} shots with {} damage'.format(shots, damage))
plt.ylabel('Probability, %')
plt.xlabel('Wounds lost')
plt.bar(damage_dealt.keys(), [i * 100 for i in damage_dealt.values()], color='g', width=1, align='center')
fig.text(0.67, 0.75, 'Expected: {:.1f}'.format(sum(key * value for key, value in damage_dealt.items())),
         fontsize=12, fontweight='bold', horizontalalignment='right')
plt.show()